# Creating MCP Client 

## Chatbot Example

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def process_query(query: str):
    messages = [{"role": "user", "content": query}]

    while True:
        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=messages,
            functions=functions,
            function_call="auto",
            max_tokens=2024
        )
        msg = resp.choices[0].message

        # 1) 일반 텍스트 응답
        if msg.content is not None:
            print(msg.content)
            break

        # 2) 함수 호출 응답
        call = msg.function_call
        fname = call.name
        fargs = json.loads(call.arguments)

        print(f"Calling tool `{fname}` with args: {fargs}")
        tool_result = execute_tool(fname, fargs)

        # assistant의 함수 호출 메시지를 기록
        messages.append({
            "role": "assistant",
            "function_call": {
                "name": fname,
                "arguments": call.arguments
            }
        })
        # 함수 실행 결과를 function 역할로 추가
        messages.append({
            "role": "function",
            "name": fname,
            "content": tool_result
        })
        # 루프 계속 → 모델이 function 결과를 바탕으로 응답 생성

def chat_loop():
    print("쿼리를 입력하거나 'quit'를 입력해 종료합니다.")
    while True:
        query = input("Query: ").strip()
        if query.lower() == "quit":
            break
        try:
            process_query(query)
        except Exception as e:
            print(f"Error: {e}")

## Building your MCP Client

### Reference Code

```python
from mcp import ClientSession, StdioServerParameters, types
from mcp.client.stdio import stdio_client

# stdio 연결을 위한 서버 파라미터 생성
server_params = StdioServerParameters(
    command="uv",  # 실행 파일
    args=["run example_server.py"],  # 커맨드 라인 인자
    env=None,  # (선택 사항) 환경 변수
)

async def run():
    # 서버를 서브프로세스로 실행하고, 읽기/쓰기 스트림을 반환함
    # read: 클라이언트가 서버로부터 메시지를 읽는 데 사용하는 스트림
    # write: 클라이언트가 서버에 메시지를 쓰는 데 사용하는 스트림
    async with stdio_client(server_params) as (read, write): 
        # 클라이언트 세션은 서버와의 연결을 초기화하고 요청을 보내는 데 사용됨
        async with ClientSession(read, write) as session:
            # 연결 초기화 (서버와의 1:1 연결)
            await session.initialize()

            # 사용 가능한 도구 목록 조회
            tools = await session.list_tools()

            # 이곳에서 chat_loop를 호출할 예정
            # ....

            # 도구 호출: 실제 호출은 process_query 메서드 안에서 이루어짐
            result = await session.call_tool("tool-name", arguments={"arg1": "value"})


if __name__ == "__main__":
    asyncio.run(run())
```

### Adding MCP Client to the Chatbot

### `mcp_chatbot` code

In [10]:
%%writefile mcp_project/mcp_chatbot.py

from dotenv import load_dotenv
import os
import json
import asyncio
import nest_asyncio

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from openai import OpenAI
import config

nest_asyncio.apply()
load_dotenv()

client = OpenAI(api_key=config.API_KEY)

class MCP_ChatBot:

    def __init__(self):
        self.session: ClientSession = None
        self.available_tools: list[dict] = []
        self.available_functions: list[dict] = []

    async def process_query(self, query: str):
        # 사용자 메시지로 대화 시작
        messages = [{"role": "user", "content": query}]

        # MCP 서버에서 받아온 도구 목록을 OpenAI 함수 스펙으로 변환
        self.available_functions = [
            {
                "name": tool["name"],
                "description": tool["description"],
                "parameters": tool["input_schema"],
            }
            for tool in self.available_tools
        ]

        while True:
            # OpenAI 최신 클라이언트 메서드 사용
            response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=messages,
                functions=self.available_functions,
                function_call="auto",
                max_tokens=2024,
                temperature=0.7
            )
            msg = response.choices[0].message

            # 모델이 함수(도구) 호출을 요청하면
            if getattr(msg, "function_call", None):
                func_name = msg.function_call.name
                func_args = json.loads(msg.function_call.arguments)
                print(f"Calling tool {func_name} with args {func_args}")
                
                # MCP 프로토콜로 도구 실행
                tool_result = await self.session.call_tool(func_name, arguments=func_args)
                
                # 함수 실행 결과를 대화에 추가
                messages.append({
                    "role": "function",
                    "name": func_name,
                    "content": tool_result.content
                })
                # 결과 반영 후 다시 반복
                continue

            # 일반 텍스트 응답이면 출력 후 종료
            print(msg.content)
            messages.append({"role": "assistant", "content": msg.content})
            
            break

    async def chat_loop(self):
        print("\nMCP 챗봇을 시작합니다!\n쿼리를 입력하거나 'quit'를 입력해 종료합니다.")
        while True:
            query = input("\nQuery: ").strip()
            if query.lower() == 'quit':
                break
            await self.process_query(query)
            print("\n")

    async def connect_to_server_and_run(self):
        # FastMCP 서버를 stdio로 실행
        server_params = StdioServerParameters(
            command="uv",
            args=["run", "research_server.py"],
            env=None,
        )
        async with stdio_client(server_params) as (read, write):
            async with ClientSession(read, write) as session:
                self.session = session
                
                # 세션 초기화 및 도구 리스트 가져오기
                await session.initialize()
                response = await session.list_tools()
                tools = response.tools
                
                print("\nConnected to server with tools:", [tool.name for tool in tools])
                self.available_tools = [
                    {"name": tool.name, "description": tool.description, "input_schema": tool.inputSchema}
                    for tool in tools
                ]
                await self.chat_loop()

async def main():
    chatbot = MCP_ChatBot()
    await chatbot.connect_to_server_and_run()

if __name__ == "__main__":
    asyncio.run(main())

Overwriting mcp_project/mcp_chatbot.py
